# Day 3


In [3]:
import re
import numpy as np

## Pt 1


In [53]:
class EngineSchematic:
    def __init__(self, filename: str):
        self.schematic = self.get_engine_schematic(filename)
        self.previous_line = None
        self.current_line = None
        self.next_line = None
        self.engine_numbers = []
        self.engine_sum = 0
        self.symbols = []
        self.checked_positions = []
        
    @staticmethod 
    def get_engine_schematic(filename: str):
        f1 = open(f'{filename}.txt', 'r')
        f1.seek(0)
        schematic = np.array([line.strip() for line in f1.readlines()])
        return schematic
    
    def process_engine_schematic(self):
        for index, line in enumerate(self.schematic, start=0):
            self.symbols = []
            self.previous_line = self.schematic[index - 1] if index > 0 else None
            self.current_line = line
            self.next_line = self.schematic[index + 1] if index < len(self.schematic) - 1 else None
            
            self.process_lines()
            
        for number in self.engine_numbers:
            self.engine_sum += number
            
        print(self.engine_sum)
            
    def process_lines(self):
        numbers = re.findall(r"-?\d+", self.current_line)
        numbers = [int(num) for num in numbers]
        
        lines = [line for line in [self.previous_line, self.current_line, self.next_line] if line is not None]
        for row_index, line in enumerate(lines):
            for column_index, char in enumerate(line):
                if char not in [".", " "] and not char.isdigit():
                    self.symbols.append((char, row_index, column_index))
        
        checked_positions = []

        for index, number in enumerate(numbers, start=0):
            top, left, bottom, right, checked_positions = self.find_bounding_region(lines, number, checked_positions)
            # print(index, number, end=" ")
            if top is not None:
                for symbol, symbol_x, symbol_y in self.symbols:
                    if top <= symbol_x <= bottom and left <= symbol_y <= right:
                        self.engine_numbers.append(number)
                        continue

                    
    @staticmethod
    def find_bounding_region(matrix, number, checked_positions):
        rows: int = len(matrix)
        cols: int = len(matrix[0])
        number_str: str = str(number)
        
        left = cols
        right = -1
        top = rows
        bottom = -1
        
        def check_number_at(i, j):
            if j + len(number_str) > cols:
                return False 
            flag = True
            for k in range(len(number_str)):
                if matrix[i][j + k] == number_str[k]:
                    flag = False
            if flag:
                return False
            
            # if j > 0 and matrix[i][j - i].isdigit():
            #     return False
            # if j + len(number_str) < cols and matrix[i][j + len(number_str)].isdigit():
            #     return False
            if (i, j) in checked_positions:
                return False
            return True
        
        for i in range(rows):
            for j in range(cols):
                if check_number_at(i, j):
                    top = min(top, i)
                    bottom = max(top, i)
                    left = min(left, j)
                    right = max(right, j + len(number_str) - 1)
                    checked_positions.append((i, j))

            
        if right == 1:
            return None
        
        left = max(0, left - 1)
        right = min(cols - 1, right + 1)
        top = max(0, top - 1)
        bottom = min(rows - 1, bottom + 1)
        
        return top, left, bottom, right, checked_positions

In [62]:
schematic_sum = EngineSchematic(filename="input")
schematic_sum.process_engine_schematic()